## Game pattern and bye week constraints

In [8]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [9]:
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

In [10]:
W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

In [11]:
#W

In [12]:
#T

In [13]:
#E

In [14]:
# Create an empty model
m2a = gp.Model(name="NFL1")

#### ADD DECISION VARIABLES 

In [30]:
#if team h plays at home against team j at week k
x = m2a.addVars(T, T, W, vtype = GRB.BINARY, name = "x")

In [31]:
#if team h plays at home in week k and k+1
y = m2a.addVars(T, W, vtype = GRB.BINARY, name = "y")

In [32]:
#if team h plays at away in week k and k+1
z = m2a.addVars(T, W, vtype = GRB.BINARY, name = "z")

#### Add Objective function

In [33]:
# Objective is to minimize the total distance of all teams
obj = sum(sum(x[i,j,k]*2*E.iloc[i,j] for k in W) for i in T for j in T)

#### ADD CONSTRAINTS HERE

In [34]:
#The season was limited to 12 weeks.
con1 = m2a.addConstrs(sum(x[i,j,k] + x[j,i,k] for k in W for i in T) == 12 
                     for j in T)

In [35]:
#each team would play once per week
con2 = m2a.addConstrs(sum(x[i,j,k] + x[j,i,k] for i in T) == 1 
                    for j in T for k in W)

In [36]:
#add constraints that all 12 games that a team played would need to be against a different opponent 
con3 = m2a.addConstrs(sum(x[i,j,k] + x[j,i,k] for k in W) <= 1
                    for i in T for j in T)

In [37]:
#add constraints that each team would play at most 6 home games
con4 = m2a.addConstrs(sum(x[i,j,k] for i in T for k in W) <= 6 for j in T)

In [38]:
#add constraints that no team can plays itself
con = m2a.addConstrs((x[i,i,k] == 0 for i in T for k in W))

In [39]:
#add constraints that no team would play more than 2 consecutive games in home
con5 = m2a.addConstrs(sum(y[i,k] for k in W) <= 1 for i in T)

In [57]:
#Implication of constraint 5
con5_i = m2a.addConstrs(sum(x[i,j,k] + x[j,i,k+1] for j in T) <= 1 + y[i,k] for k in set(list(W)[:-1]) for i in T)

In [58]:
#add constraints that no team would play more than 2 consecutive games in away
con6 = m2a.addConstrs(sum(z[i,k] for k in W) <= 1 for i in T)

In [60]:
#Implication of constraint 6
con6_i = m2a.addConstrs(sum(x[i,j,k] + x[j,i,k+1] for j in T) <= 1 + z[i,k] for k in set(list(W)[:-1]) for i in T)

#### Optimization

In [ ]:
m2a.setObjective(obj, GRB.MINIMIZE)
m2a.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 3528 rows, 14688 columns and 148224 nonzeros
Model fingerprint: 0x26f9d02b
Variable types: 0 continuous, 14688 integer (14688 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 2364 rows and 7536 columns
Presolve time: 0.22s
Presolved: 1164 rows, 7152 columns, 51840 nonzeros
Variable types: 0 continuous, 7152 integer (7152 binary)

Root relaxation: objective 1.659080e+05, 5598 iterations, 1.12 seconds
Total elapsed time = 5.90s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  474          - 165908.000      -     -    6s
     0     0 165908.000    0  551          - 165908.000      -     -    7s
     0     0 165908.000    0  538          - 165908.000

 59391 49516 166267.483  221  413          - 165908.000      -   278 1517s
 60115 50071 166083.660  210  406          - 165908.000      -   278 1541s
 60740 50874 166109.257  213  387          - 165908.000      -   280 1568s
 61690 51836 infeasible  284               - 165908.000      -   281 1594s
 62802 52863 166615.142  251  409          - 165908.000      -   281 1621s
 63975 53642 169931.957  199  425          - 165908.000      -   281 1646s
 64912 54587 167017.381  209  337          - 165908.000      -   281 1672s
 65998 55199 165913.911  235  380          - 165908.000      -   282 1697s
 66703 55792 166774.840  333  370          - 165908.000      -   283 1723s
 67509 56696 166809.389  279  404          - 165908.000      -   284 1748s
 68507 57241 166479.232  220  395          - 165908.000      -   284 1773s
 69265 58132 168028.982  262  392          - 165908.000      -   285 1799s
 70318 58975 175896.044  423  326          - 165908.000      -   286 1824s
 71371 59735 166358.997  

 155294 130309 168895.518  233  366          - 165908.000      -   321 4117s
 156039 131092 167684.135  314  370          - 165908.000      -   322 4138s
 156930 132719 169035.008  365  374          - 165908.000      -   322 4163s
 158755 133386 167228.999  196  369          - 165908.000      -   320 4188s
 159570 134319 167992.471  208  381          - 165908.000      -   320 4219s
 160657 135417 infeasible  325               - 165908.000      -   319 4246s
 161875 136297 168751.922  234  344          - 165908.000      -   319 4269s
 162856 137058 infeasible  361               - 165908.000      -   318 4293s
 163813 137680 168555.375  274  334          - 165908.000      -   318 4319s
 164541 138340 167377.686  215  343          - 165908.000      -   318 4345s
 165357 139116 172795.500  219  293          - 165908.000      -   318 4373s
 166255 139645 165908.000   60  570          - 165908.000      -   318 4402s
 166894 140649 165908.000  172  492          - 165908.000      -   318 4427s

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\yuezh\conda\lib\site-packages\ipykernel\iostream.py", line 387, in write
    if self.echo is not None:
KeyboardInterrupt


 172581 145650 168148.253  290  295          - 165908.000      -   315 4540s
 173629 146711 168289.381  222  327          - 165908.000      -   314 4554s
 174814 147459 169819.951  288  310          - 165908.000      -   314 4570s
 175752 148179 168764.824  244  338          - 165908.000      -   313 4585s
 176578 149120 167741.473  224  325          - 165908.000      -   313 4602s
 177595 149855 169805.101  341  332          - 165908.000      -   313 4618s
 178488 150834 167155.847  228  410          - 165908.000      -   312 4637s
 179545 151409 167768.772  281  392          - 165908.000      -   312 4655s
 180241 152005 168426.193  185  373          - 165908.000      -   312 4672s
 180927 152559 169230.000  210  268          - 165908.000      -   312 4688s
 181587 153049 167571.597  200  357          - 165908.000      -   312 4705s
 182177 153645 infeasible  220               - 165908.000      -   312 4726s
 182861 154289 167901.590  227  355          - 165908.000      -   312 4748s

In [ ]:
# ADD PRINTING HERE:
# print the total distance travelled by all teams

if m2a.status == GRB.OPTIMAL:
    # Print solution
    print("\nTotal distance: {}\n". format(m2a.ObjVal))    

# print the optimal schedule for the decision variables

print('\nDecision variables:')
for v in m2a.getVars():
    if v.x== 1:
        s=("%s : %g" % (v.varName, v.obj))
        print(s)

#### print the total distance travelled by all teams

In [23]:
print('\nValue of objective function: %g' % m1.objVal)


Value of objective function: 165908


In [24]:
# print the optimal schedule for Cleveland Browns, 14

In [25]:
x[3,14,1] : 340
x[7,14,5] : 912
x[14,2,4] : 688
x[14,4,2] : 1100
x[14,5,10] : 1504
x[14,11,6] : 748
x[14,12,8] : 370
x[14,13,11] : 500
x[16,14,7] : 630
x[19,14,0] : 1278
x[20,14,9] : 920
x[21,14,3] : 266